Update February 2022 --

This is a notebook from my Capstone Rework project; [https://github.com/np1919/Capstone-Rework](https://github.com/np1919/Capstone-Rework). I've left the description at the top to give some context; the class itself could definitely use some work.

In the process of eventually deploying an app to `Heroku`, I've come a long way in terms of organizing my ETL structures. I'm grateful for having spent time learning more about coding in Python. This class is far too expensive, computationally, to run on a remote server. 

That limitation has forced me to consider alternatives -- and structure my 'business outcomes' in a more readable way. I'm going to go back and create several spectra of class labels; and then compare modelling results for each, such that I can find the households most similar to one another in certain areas -- this process in the end may involve more market basket analysis; fpgrowth; and association rules analysis. I'm figuring it out, albeit slowly. 

In the end I hope to be able to provide several core recommendation tables; for each of 3-5 class labels for households in the data. Moreover; to be able to generate those class labels using new data that might become available.

# DunnHumby Recommender System 

Because of the complexity of this dataset, and my weak background in statistics, I opted to showcase a Python class structure capable of generating recommendations for any given household based on their previous purchases. 

My thinking is that a system like this would be able to effectively identify profitable spending patterns by a household -- those associated products which influence the customer to spend more or purchase more frequently. 

For example, if our competitor pricing research found that we should target a specific product in order to vie for market share; **the recommender system below would be able to identify what other products might do well alongside the 'targeted' item in a marketing campaign to that customer. With multiple reasons to come to the store, the customer might be more inclined to make the trip, or to spend more if they do**. 

I recognize that in a brick-and-mortar store, having live-updating recommendations based on items in basket is not necessarily as useful as it would be in an online shop -- however, I believe there is value to be offered by using a system like this.

Data Pipeline
---

The concept (and practice!) of molding data into a 'pipeline' is a deep one -- the 'correct answer' has to be negotiated based on current data infrastructure, available resources and future scope.

An effective data pipeline would:

- generate or otherwise acquire data
- ensure the data's cleanliness and integrity
- seamlessly add the data to an existing data pool
- access that data as needed, for analysis and modelling
- surface that analysis and modelling results

For the scope of this project, we don't have much of a chance to acquire more data. But, if we assume that any other data would be in the same format as the data we've already received (big leap), then we've already made some work on creating a pipeline for future data.

**pipeline thus far**:
- commodity breakdown/sales analysis
- adding datetime
- data truncation
- customer segmentation (RFM score)


The pipeline is important, as it allows us to quickly identify changing patterns in customer purchase behaviour in the case of live or newly acquired data in the same format.

Adding datetime is a 'housekeeping' issue which makes things easier on our end. 

Customer segmentation, however, provides some valuable insight about any given customer (household) in our data. 

By segmenting the customers in various ways, we can create effective recommendations for a given subset of our customers -- for example, the most profitable ones. 

In [48]:
#import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (16,6)
import datetime

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth

import my_funcs


Forming `merged` from `product.csv` and `transaction_data.csv`
---

In [65]:
# import transactions
# import product info
# join commodity_desc

transactions = pd.read_csv('data/transaction_data.csv')
product = pd.read_csv('data/product.csv')
merged = transactions.merge(product[['PRODUCT_ID', 'COMMODITY_DESC']])


In [66]:
# calling my module functions to map Section Labels and Datetime

merged['Section Labels'] = my_funcs.return_section_labels(merged)

my_funcs.add_datetime(merged)

Confirming christmas 1 is at DAY 278: 2004-12-25 00:00:00


In [67]:
my_funcs.add_datetime(merged)

Confirming christmas 1 is at DAY 278: 2004-12-25 00:00:00


In [68]:
merged['datetime']

0         2004-03-23 16:31:00
1         2004-03-26 17:55:00
2         2004-03-26 00:35:00
3         2004-03-27 15:51:00
4         2004-03-29 23:03:00
                  ...        
2595727   2006-03-03 15:20:00
2595728   2006-03-03 15:20:00
2595729   2006-03-03 15:20:00
2595730   2006-03-03 15:20:00
2595731   2006-03-03 15:20:00
Name: datetime, Length: 2595732, dtype: datetime64[ns]

In [69]:
# confirm each basket has a unique datetime index..

all(merged.groupby('BASKET_ID')['datetime'].nunique()) == 1

True

# Recommender System for Households
---

Using FPGrowth (apriori and association rules)

Below I've implemented a rudimentary (minimum viable product?) Recommender System class.

There is so much left for me to learn about how a class like this might be integrated into 'production'.


In [75]:
class RecommenderSystem:
    '''
    ## hh_key :  the household_key
    ## df : the transactions df; 
    ## column : the column in df to be used for MBA ('COMMODITY_DESC, DEPARTMENT, SUB_COMMODITY_DESC')
    ## max_len :  max length of any antecedent/consequent chains in support_table
    ## support_threshold : minimum 'support' threshold to generate fpgrowth
    
    ## metric : the association rules metric to maximize
    ## assoc_threshold : the association rules threshold, given the metric.
    '''
        ## Instantiate Class

    def __init__(self, 
                 hh_key,   ### FOR HOUSEHOLDS#!### 
                 df=merged, 
                 column='COMMODITY_DESC', 
                    max_len=None, ### CONSIDER REDUCING THIS VALUE FOR SIMPLICITY ###
                 support_threshold=0.05, ### WITH DATA OF FIXED SIZE, NOT A CONCERN? ###
                metric='confidence', 
                 assoc_threshold=0.8,
                ):
                                    #TODO: implement intelligent thresholds
        self.hh = hh_key
        self.metric = metric
        self.assoc_threshold=assoc_threshold
        self.column = column
        self.support_threshold = support_threshold
        self.df = df[df['household_key'] == self.hh] # self.df is transactions for this customer only
        self.max_len = max_len
        

        # create support table upon instantiation
        
        self.get_support_table()
     
    
    ### Support Table Function ###
    # uses fpgrowth to generate a support table
    def get_support_table(self):
        '''Return the support table for `BASKET_ID`s using `column` as product lists
        Note: 'BASKET_ID' is hardcoded...
        
        '''
        # create product lists for each basket                                   
        product_lists = self.df.groupby('BASKET_ID')[self.column].apply(list) # apply list constructor
 
        # dummy encoding...
        te = TransactionEncoder()
        te_fit = te.fit_transform(product_lists.values, sparse=True) # encode each 
        te_df = pd.DataFrame.sparse.from_spmatrix(te_fit, columns=[str(i) for i in te.columns_])
      
        # fpgrowth table
        frequent_itemsets = fpgrowth(te_df, 
                                    min_support=self.support_threshold, #can alter self.support_threshold
                                    use_colnames=True, 
#                                     verbose=True, 
                                    max_len=self.max_len,   # can alter self.max_len here.
                                    #, low_memory=True,                                   
                                    )
        # adding a length column for posterity and filtering
        frequent_itemsets['size'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
        
        # save variable for reference...
        self.support_table = frequent_itemsets

    
    @property
    def assoc_table(self):
        '''change self.metric, self.assoc_threshold to rank differently'''
        ##  calling association rules on our support table
        rules = association_rules(self.support_table, metric=self.metric, min_threshold=self.assoc_threshold)
        rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
        return rules
    
    def recommend(self, prev_purchases:list, howmany=5):
        '''meat and bones of the recommender system...
        accepts:
            prev_purchases: a list of previously purchased items
            howmany: (int) how many recommendations you want
            
        returns:
            a series consisting of the top 5 results given the self.metric value.
            '''
        search_terms = list(prev_purchases) # handles frozensets?
        # apply list to 'antecedent' in self.assoc_table
        search_series = pd.Series(self.assoc_table['antecedents'].apply(list)) 
        
        print(f'Searching for {search_terms}...')
        indexes_of_matches = []

        # for each antecedent chain...
        for item in search_terms:
            # iterate through the list of "antecedent" rows **search_series**
            for idx, val in search_series.iteritems():
                if item in val: # if the item is in the row (list of antecedents)..
                    indexes_of_matches.append(idx)

        rules = self.assoc_table.loc[indexes_of_matches]


        # RETURN TOP 5 LIFT CONSEQUENTS
        return rules.sort_values(self.metric, ascending=False)[:howmany]['consequents']

## Demonstration
---

For `household_key == 2`, looking at 'confidence':

In [76]:
hh2 = RecommenderSystem(hh_key=2, metric='confidence', assoc_threshold=0.8)#, max_len=2)

print(hh2.metric)
hh2.recommend(['APPLES'])
# returns the top 5 'lift' of all purchases with 'APPLES' antecedent. 

confidence
Searching for ['APPLES']...


1429                  (BAKED BREAD/BUNS/ROLLS)
185415                (BAKED BREAD/BUNS/ROLLS)
185403                 (YOGURT, COOKIES/CONES)
185405    (BAKED BREAD/BUNS/ROLLS, MARGARINES)
185406        (YOGURT, BAKED BREAD/BUNS/ROLLS)
Name: consequents, dtype: object

Looking at 'lift'

In [77]:
hh2 = RecommenderSystem(hh_key=2, metric='lift', assoc_threshold=0.8)#, max_len=2)
print(hh2.metric)
hh2.recommend(['APPLES'])
# returns the top 5 'lift' of all purchases with 'APPLES' antecedent.

lift
Searching for ['APPLES']...


363522    (FRUIT - SHELF STABLE, CRACKERS/MISC BKD FD, F...
385436                  (YOGURT, COOKIES/CONES, MARGARINES)
386339    (CANNED JUICES, BAKED BREAD/BUNS/ROLLS, COOKIE...
385501         (YOGURT, BAKED BREAD/BUNS/ROLLS, MARGARINES)
385498                  (YOGURT, COOKIES/CONES, MARGARINES)
Name: consequents, dtype: object

Looking at 'conviction':

In [78]:
hh2.metric='conviction'
hh2.recommend(['APPLES']) 

Searching for ['APPLES']...


222012                         (SOFT DRINKS, COOKIES/CONES)
363637    (FRUIT - SHELF STABLE, BAKED BREAD/BUNS/ROLLS,...
363576    (FRUIT - SHELF STABLE, SOFT DRINKS, FLUID MILK...
363583    (CRACKERS/MISC BKD FD, SOFT DRINKS, FLUID MILK...
386400            (YOGURT, MARGARINES, FLUID MILK PRODUCTS)
Name: consequents, dtype: object

Interpretation
---

For multi-item antecedent chains (items in 'basket'), this recommender searches for recommendations for each item independently, then concatenates the full table of results and sorts by the metric currently selected in the class .metric attribute.

So many things to consider, with a technology like this. Does it make sense to filter more stringently, and only look for chains of antecedents precisely equal to the search term? What about finding similar products using something like word2vec or other NLP? How do we change or vary our recommendations, so that customers aren't constantly seeing the same recommendations?

We could use cluster labels, RFM score, demographics to populate a 'similar customer' association rules table, and make recommendations from there. A basic approach would be to recommend based on recently purchased items; or to reverse-engineer this process and recommend 'trigger' products which we might want to be selling to them.

We're getting redundant values in our consequent chains when parsing 5 straight from the top of the `confidence` metric. What about returning the results from a blend of metrics; another option would be to add another layer of logic to prevent redundant recommendations. (create a 'set', and don't stop filling it until it reaches 5 items). 

One issue about having an antecedent 'chain' instead of a single result: the recommendations won't always be obviously related. This could be frustrating for a customer. 


Run the recommender on all of the purchases by a household using the support table of similarly-labeled customers.

Alternative Methods of Recommendation
---


- Instead of using the fpgrowth table for each household proper; use the Cluster label (or RFM score group) to generate a 'similar-customers' fpgrowth table; call .recommend() using that table, for the most recent purchases by the household.

- Search a customer's history to see their 'DEPARTMENT' or 'COMMODITY' spending vs the average? 

- Get the rows for each top ranked value in a given support table.. #TODO: deal with ties, inf 

- Use NLP or regex to find similar items and make the search terms more robust.

## Second Example

Expanding on the functionality
---


By altering the last few lines of the recommend() function, we could instead return a `set` of items from the top of our results; or add several recommender metrics together and return the set of all items. for example:

In [81]:
class RecommenderSystemSET:
    '''
    ## hh_key :  the household_key
    ## df : the transactions df; 
    ## column : the column in df to be used for MBA ('COMMODITY_DESC, DEPARTMENT, SUB_COMMODITY_DESC')
    ## max_len :  max length of any antecedent/consequent chains in support_table
    ## support_threshold : minimum 'support' threshold to generate fpgrowth
    
    ## metric : the association rules metric to maximize
    ## assoc_threshold : the association rules threshold, given the metric.
    '''
        ## Instantiate Class

    def __init__(self, 
                 hh_key,   ### FOR HOUSEHOLDS#!### 
                 df=merged, 
                 column='COMMODITY_DESC', 
                    max_len=None, ### CONSIDER REDUCING THIS VALUE FOR SIMPLICITY ###
                 support_threshold=0.05, ### WITH DATA OF FIXED SIZE, NOT A CONCERN? ###
                metric='confidence', 
                 assoc_threshold=0.8,
                ):
                                    #TODO: implement intelligent thresholds
        self.hh = hh_key
        self.metric = metric
        self.assoc_threshold=assoc_threshold
        self.column = column
        self.support_threshold = support_threshold
        self.df = df[df['household_key'] == self.hh] # self.df is transactions for this customer only
        self.max_len = max_len
        

        # create support table upon instantiation
        
        self.get_support_table()
     
    
    ### Support Table Function ###
    
    # uses fpgrowth to generate a support table
    
    def get_support_table(self):
        '''Return the support table for `BASKET_ID`s using `column` as product lists
        Note: 'BASKET_ID' is hardcoded...
        
        '''
        # create product lists for each basket                                   
        product_lists = self.df.groupby('BASKET_ID')[self.column].apply(list) # apply list constructor
 
        # dummy encoding...
        te = TransactionEncoder()
        te_fit = te.fit_transform(product_lists.values, sparse=True) # encode each 
        te_df = pd.DataFrame.sparse.from_spmatrix(te_fit, columns=[str(i) for i in te.columns_])
      
        # fpgrowth table
        frequent_itemsets = fpgrowth(te_df, 
                                    min_support=self.support_threshold, #can alter self.support_threshold
                                    use_colnames=True, 
#                                     verbose=True, 
                                    max_len=self.max_len,   # can alter self.max_len here.
                                    #, low_memory=True,                                   
                                    )
        # adding a length column for posterity and filtering
        frequent_itemsets['size'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
        
        # save variable for reference...
        self.support_table = frequent_itemsets

    
    @property
    def assoc_table(self):
        '''change self.metric, self.assoc_threshold to rank differently'''
        ##  calling association rules on our support table
        rules = association_rules(self.support_table, metric=self.metric, min_threshold=self.assoc_threshold)
        rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
        return rules
    
    
    def recommend(self, prev_purchases:list, howmany=5):
        '''Edited Function
            '''

        search_terms = list(prev_purchases) # handles frozensets?
        # apply list to 'antecedent' in self.assoc_table
        search_series = pd.Series(self.assoc_table['antecedents'].apply(list)) 

        print(f'Searching for {search_terms}...')
        indexes_of_matches = []

        # for each antecedent chain...
        for item in search_terms:
            # iterate through the list of "antecedent" rows **search_series**
            for idx, val in search_series.iteritems():
                if item in val: # if the item is in the row (list of antecedents)..
                    indexes_of_matches.append(idx)

        rules = self.assoc_table.loc[indexes_of_matches]

        # RETURN TOP 5 LIFT CONSEQUENTS
        results = []
        
        
        # TODO: check if the tables are populating correctly given only self.metric...
        for metric in ['confidence', 'conviction', 'lift', 'leverage', 'support']:
            results.extend(list(rules.sort_values(metric, ascending=False)[:howmany]['consequents']))

        set_results = []
        for x in results:
            set_results.extend(list(x))
        #       
        return set(set_results)


In [82]:
hh2 = RecommenderSystemSET(2)

In [85]:
set(merged[merged['household_key']==2]['COMMODITY_DESC']) # all previous purchases

{'ADULT INCONTINENCE',
 'AIR CARE',
 'ANALGESICS',
 'APPLES',
 'BABY FOODS',
 'BACON',
 'BAG SNACKS',
 'BAKED BREAD/BUNS/ROLLS',
 'BAKED SWEET GOODS',
 'BAKING',
 'BAKING MIXES',
 'BAKING NEEDS',
 'BATH',
 'BATH TISSUES',
 'BATTERIES',
 'BEEF',
 'BEERS/ALES',
 'BERRIES',
 'BLEACH',
 'BREAD',
 'BREAKFAST SAUSAGE/SANDWICHES',
 'BREAKFAST SWEETS',
 'BROCCOLI/CAULIFLOWER',
 'BROOMS AND MOPS',
 'CANDY - CHECKLANE',
 'CANDY - PACKAGED',
 'CANNED JUICES',
 'CARROTS',
 'CHEESE',
 'CHEESES',
 'CHICKEN',
 'CHRISTMAS  SEASONAL',
 'CIGARETTES',
 'CITRUS',
 'COFFEE',
 'COFFEE FILTERS',
 'COLD CEREAL',
 'CONDIMENTS/SAUCES',
 'CONVENIENT BRKFST/WHLSM SNACKS',
 'COOKIES/CONES',
 'COOKWARE & BAKEWARE',
 'CORN',
 'COUPON/MISC ITEMS',
 'CRACKERS/MISC BKD FD',
 'DELI MEATS',
 'DEODORANTS',
 'DINNER MXS:DRY',
 'DISHWASH DETERGENTS',
 'DISPOSIBLE FOILWARE',
 'DOG FOODS',
 'DRIED FRUIT',
 'DRY BN/VEG/POTATO/RICE',
 'DRY MIX DESSERTS',
 'DRY NOODLES/PASTA',
 'EGGS',
 'ELECTRICAL SUPPPLIES',
 'FACIAL TISS/DNR 

In [86]:
# most frequent????

In [88]:
most_recent = list(merged[merged['household_key']==2]['COMMODITY_DESC'])[-5:]
most_recent

['PET CARE SUPPLIES',
 'HAND/BODY/FACIAL PRODUCTS',
 'DOG FOODS',
 'CANDY - PACKAGED',
 'PET CARE SUPPLIES']

In [89]:
hh2.metric = 'lift'

In [90]:
hh2.recommend(most_recent)

Searching for ['PET CARE SUPPLIES', 'HAND/BODY/FACIAL PRODUCTS', 'DOG FOODS', 'CANDY - PACKAGED', 'PET CARE SUPPLIES']...


{'BAKED BREAD/BUNS/ROLLS',
 'BAKED SWEET GOODS',
 'BAKING MIXES',
 'BAKING NEEDS',
 'BEEF',
 'COOKIES/CONES',
 'DEODORANTS',
 'DOG FOODS',
 'FLUID MILK PRODUCTS',
 'PET CARE SUPPLIES',
 'POTATOES',
 'SOFT DRINKS',
 'VEGETABLES - SHELF STABLE'}

In [91]:
hh2.recommend(['BAKED BREAD/BUNS/ROLLS'])

Searching for ['BAKED BREAD/BUNS/ROLLS']...


{'APPLES',
 'BAKED SWEET GOODS',
 'BAKING MIXES',
 'BAKING NEEDS',
 'BEEF',
 'CANNED JUICES',
 'COOKIES/CONES',
 'FLUID MILK PRODUCTS',
 'MARGARINES',
 'ONIONS',
 'PAPER HOUSEWARES',
 'POTATOES',
 'SOFT DRINKS',
 'SUGARS/SWEETNERS',
 'VEGETABLES - SHELF STABLE',
 'YOGURT'}

In [92]:
hh5 = RecommenderSystem(hh_key=5, support_threshold=0.1) # consider opening this range further
hh5.metric='confidence'
hh5.assoc_threshold=0.2 # very low. 

hh5.assoc_table

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(BAG SNACKS),(SALADS/DIPS),0.150,0.175,0.1,0.666667,3.809524,0.07375,2.475000,1
1,(SALADS/DIPS),(BAG SNACKS),0.175,0.150,0.1,0.571429,3.809524,0.07375,1.983333,1


# Association Rules Metrics 

There is a ton of information that can be gathered by this sort of analysis;

By calculating the 'frequency of occurrence' (`support`) for each item (in this case, `COMMODITY_DESC`) as such, we can then perform calculations to determine **if the presence of one product in the purchase basket** has affected the chance that **another product will be purchased** ; by how much (`lift` factor); and how strong the association is (`confidence` proportion).

Below we can see some of the calculations quickly using the `association_rules` function:


For example, one top-rated association in the `lift` category from the table above is [butter and pasta sauce] -> [dry noodles and cheese]. This means that the chance of having dry noodles AND cheese in your basket -- if you have butter AND pasta sauce *already* -- is increased by a factor of almost 16 times compared to the original `supports` of those chains of items independent of one another. 

If instead we sort the table by `confidence`, we can look at the proportion of co-occurence the `antecedent` (chain) exists on it's own.

We create this table below using a threshold for the `confidence` metric; calculated as: how many baskets does the `consequent` appear in **with** the `antecedent`, divided by the total number of baskets in which the `antecedent` is present.

A confidence score approaching 1 reflects that the `consequent` is found in the `antecedent`'s baskets
approaching --> 100% of the time. 

Below we can see the highest-`confidence` relationships from the the purchases in our data, and the categories in `COMMODITY_DESC`.

In [93]:
hh2 = RecommenderSystem(2)
hh2.metric = 'confidence'
hh2.assoc_table.sort_values(hh2.metric, ascending=False).head(12)
#This table changes slightly without max_len=4

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
169500,"(CANNED JUICES, BAKED BREAD/BUNS/ROLLS, BAKING...","(POTATOES, FLUID MILK PRODUCTS, VEGETABLES - S...",0.066667,0.088889,0.066667,1.0,11.25,0.060741,inf,3
225623,"(YOGURT, BAKING NEEDS)","(BAKING MIXES, FLUID MILK PRODUCTS, BAKED BREA...",0.066667,0.088889,0.066667,1.0,11.25,0.060741,inf,2
225621,"(YOGURT, BAKING NEEDS, BEEF)","(BAKING MIXES, FLUID MILK PRODUCTS, BAKED BREA...",0.066667,0.111111,0.066667,1.0,9.00,0.059259,inf,3
225620,"(YOGURT, SOFT DRINKS, BEEF)","(BAKING MIXES, FLUID MILK PRODUCTS, BAKED BREA...",0.066667,0.088889,0.066667,1.0,11.25,0.060741,inf,3
225619,"(YOGURT, SOFT DRINKS, BAKING NEEDS)","(BAKING MIXES, FLUID MILK PRODUCTS, BAKED BREA...",0.066667,0.088889,0.066667,1.0,11.25,0.060741,inf,3
225618,"(YOGURT, COOKIES/CONES, BEEF)","(BAKING MIXES, FLUID MILK PRODUCTS, BAKED BREA...",0.066667,0.111111,0.066667,1.0,9.00,0.059259,inf,3
225617,"(YOGURT, BAKING NEEDS, COOKIES/CONES)","(BAKING MIXES, FLUID MILK PRODUCTS, BAKED BREA...",0.066667,0.111111,0.066667,1.0,9.00,0.059259,inf,3
225616,"(YOGURT, BAKED BREAD/BUNS/ROLLS, BEEF)","(BAKING MIXES, FLUID MILK PRODUCTS, COOKIES/CO...",0.066667,0.088889,0.066667,1.0,11.25,0.060741,inf,3
225615,"(YOGURT, BAKED BREAD/BUNS/ROLLS, BAKING NEEDS)","(BAKING MIXES, FLUID MILK PRODUCTS, COOKIES/CO...",0.066667,0.088889,0.066667,1.0,11.25,0.060741,inf,3
225614,"(YOGURT, BEEF, FLUID MILK PRODUCTS)","(BAKING MIXES, BAKED BREAD/BUNS/ROLLS, COOKIES...",0.066667,0.088889,0.066667,1.0,11.25,0.060741,inf,3
